### Build Reproducible Experiments
In this notebook, the aim is to show you how to build a reproducible experiment within Azure. In this example, we're going to train a MNIST classifier using Tensorflow and show you how you can follow some practices to make this work reproducible. Later you can use this framework and apply it to other ML problems.

The reason we chose MNIST is to pick a very simple example as the main focus is to build a reproducible experiment and not to learn a new algorithm or to build a complex model.

By the end of this repo, you'll be able to build reproducible experiments based on the below image.
![Reproducible Experiments for Machine Learning](assets/reproducible_experiment_azure_machine_learning.jpg)

First we download the MNIST sample files from Yann Lecun website to our development environment:

In [ ]:
import os
import urllib

os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')



Import a bunch of packages.

In [ ]:
%matplotlib inline
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from azureml.widgets import RunDetails


Import the AzureML SDK package to be able to communicate with Azure ML Services

In [ ]:
# Check core SDK version number
import azureml.core
from azureml.core import Experiment

print("SDK version:", azureml.core.VERSION)


Initiate an object from Workspace class. the Workspace object will point to the created Workspace we created through the portal

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')


In [ ]:
# # Run this section if you're running this from outside of Compute Instance
# # import the Workspace class and check the azureml SDK version
# # exist_ok checks if workspace exists or not.

# from azureml.core import Workspace
# 

# # Your subscription ID will be different replace the stirng with yours
# subscription_id = "your_subscription_id"
# resource_group = "your_resource_group"
# workspace_name = "your_workspace_name"
# workspace_region = "your_workspace_region"

# ws = Workspace(workspace_name = workspace_name,
#                subscription_id = subscription_id,
#                resource_group = resource_group)
# 
# # persist the subscription id, resource group name, and workspace name in aml_config/config.json.
# ws.write_config()

### Start a new Experiment

In software engineering world we have a new feature to develop. In ML/Data Science world, we work on experiments.

**Experiments** represent the collection of trials used to validate a user's hypothesis. We call each trial a run.

Here we create a new experiment, we want to make sure everything related to this experiment is saved within the workspace and the lineage between each artifact is preserved.

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='at-rogers')

### 1. Track and Log Metrics

We have two ways to start a trail (Run). Interactively and through batch submite. The start_loggin() method is the interactive way of starting a trail. It returns a Run object that we can use to log important metrics or the trail.

MLFlow is natively supported in Azure Machine Learning. For examples on MLFlow integration, take a look at [[using-mlflow](https://github.com/Azure/MachineLearningNotebooks/blob/bda592a236eaf2dbc54b394e1fa1b539e0297908/how-to-use-azureml/ml-frameworks/using-mlflow)].

The key metrics can a single value for the accuracy of an ML model, a list of values representing the distribution or the data or an image showing the confusion matrix.

In [ ]:
# Initializing log tracking action by creating a Run object in the Experiment
run = exp.start_logging()

In [ ]:
run.get_status()

Here you see that the **azureml.git.repository_uri** is poinint to the remote repo and the **azureml.git.branch** property is poining to the active branch.

In [ ]:
run.get_details()

In [ ]:
from azureml.core import Run
# run = Run(experiment=exp, run_id='ca910c28-d9fe-4c94-bc55-6ed76a015f82')

In [ ]:
run.get_metrics()

**RunDetails** class helps you visualize the active run object. It creates a network connection with Azure ML Worspace to collect everything happening during the run. It gets updated every 15 seconds.

In [ ]:
RunDetails(run).show()

**logs** function will log a single-valued or multi-valued metric under the current RUN. There are several types of logs, metric, table, row, image, etc.

Every time you add a new metric, check the widget above:

In [ ]:
run.log('metric_1', 1.1)

In [ ]:
run.log('metric_1', 2.0)

In [ ]:
run.log('metric_1', 2.0)

In [ ]:
run.log('metric_1', 4.0)

In [ ]:
run.log('metric_1', 4.0)

In [ ]:
run.log('metric_1', 2.5)

In [ ]:
run.log('hossein', 6.3)

In [ ]:
# Log some metrics about the input dataset:

from utils import load_data

# Unzipping the input dataset and conver the data points into Numpy arrays
# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster.
X_train = load_data('./data/mnist/train-images.gz', False) / 255.0
y_train = load_data('./data/mnist/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/mnist/test-images.gz', False) / 255.0
y_test = load_data('./data/mnist/test-labels.gz', True).reshape(-1)

In [ ]:
# Record how the input dataset looks like
count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
    plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
    
run.log_image(name='{}-samples-of-input-dataset'.format(sample_size), plot=plt)

In [ ]:
import pandas as pd
dist = pd.DataFrame(data=y_train, columns=['test_values'])['test_values'].value_counts()

run.log_table('digit_dist', {"count":list(dist.values), "digits":list(dist.index)})

In [ ]:
X_train.shape

In [ ]:
run.log('train_dataset_size', X_train.shape[0])

In [ ]:
run.log('test_dataset_size', X_test.shape[0])

Get the collected metrics:

In [ ]:
print('metric_1: ', run.get_metrics('metric_1'))
run.get_metrics()

In [ ]:
run.complete()

### 1.1 Track local model training

In [ ]:
run = exp.start_logging()
RunDetails(run).show()

In [ ]:
import numpy as np
import argparse
import os
import re
import tensorflow as tf
import time
import glob

from azureml.core import Run
from utils import load_data
from tensorflow.keras import Model, layers

In [ ]:
training_set_size = X_train.shape[0]

n_inputs = 28 * 28
n_h1 = 100
n_h2 = 100
n_outputs = 10
learning_rate = 0.01
n_epochs = 20
batch_size = 50

run.log_row('Hyper Parameters',
        n_inputs=n_inputs,
        n_h1=n_h1,
        n_h2=n_h2,
        n_outputs=n_outputs,
        learning_rate=learning_rate,
        n_epochs=n_epochs,
        batch_size=batch_size)

In [ ]:
# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First hidden layer.
        self.h1 = layers.Dense(n_h1, activation=tf.nn.relu)
        # Second hidden layer.
        self.h2 = layers.Dense(n_h2, activation=tf.nn.relu)
        self.out = layers.Dense(n_outputs)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.h1(x)
        x = self.h2(x)
        x = self.out(x)
        if not is_training:
            # Apply softmax when not training.
            x = tf.nn.softmax(x)
        return x


def cross_entropy_loss(y, logits):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    # Average loss across the batch.
    return tf.reduce_mean(loss)


# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)


# Optimization process.
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        logits = neural_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(y, logits)

    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))


In [ ]:
# Build neural network model.
neural_net = NeuralNet()

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
start_time = time.perf_counter()
for epoch in range(0, n_epochs):

    # randomly shuffle training set
    indices = np.random.permutation(training_set_size)
    X_train = X_train[indices]
    y_train = y_train[indices]

    # batch index
    b_start = 0
    b_end = b_start + batch_size
    for _ in range(training_set_size // batch_size):
        # get a batch
        X_batch, y_batch = X_train[b_start: b_end], y_train[b_start: b_end]

        # update batch index for the next batch
        b_start = b_start + batch_size
        b_end = min(b_start + batch_size, training_set_size)

        # train
        run_optimization(X_batch, y_batch)

    # evaluate training set
    pred = neural_net(X_batch, is_training=False)
    acc_train = accuracy(pred, y_batch)

    # evaluate validation set
    pred = neural_net(X_test, is_training=False)
    acc_val = accuracy(pred, y_test)

    # log accuracies
    run.log('training_acc', np.float(acc_train))
    run.log('validation_acc', np.float(acc_val))
    print(epoch, '-- Training accuracy:', acc_train, '\b Validation accuracy:', acc_val)

    # Save checkpoints in the "./outputs" folder so that they are automatically uploaded into run history.
    checkpoint_dir = './outputs/'
    checkpoint = tf.train.Checkpoint(model=neural_net, optimizer=optimizer)

    if epoch % 2 == 0:
        checkpoint.save(checkpoint_dir)
    time.sleep(3)

In [ ]:
run.log('final_acc', np.float(acc_val))
os.makedirs('./outputs/model', exist_ok=True)

# files saved in the "./outputs" folder are automatically uploaded into run history
# this is workaround for https://github.com/tensorflow/tensorflow/issues/33913 and will be fixed once we move to >tf2.1
neural_net._set_inputs(X_train)
tf.saved_model.save(neural_net, './outputs/model/')

stop_time = time.perf_counter()
training_time = (stop_time - start_time) * 1000
print("Total time in milliseconds for training: {}".format(str(training_time)))
run.log('runtime ms', np.float(training_time))


In [ ]:
run.complete()

## 2- Working with Models
### 2-1 Register Model

In [ ]:
run.register_model('MNIST_TF_rogers', model_path='outputs/model/')

In [ ]:
from azureml.core import Model

In [ ]:
model_list = Model.list(ws)
[x for x in model_list if x.name == 'MNIST_TF_rogers']

### 2-2 Retrieve registered models

In [ ]:
model = Model(ws, 'MNIST_TF_rogers')

In [ ]:
os.makedirs('downloaded_model_2')

In [ ]:
model.download('./downloaded_model_2')

In [ ]:
model = tf.keras.models.load_model('./downloaded_model_2/model')
model.summary()

In [ ]:
y_hat = model.predict(X_test)
acc_val = accuracy(y_hat, y_test)
print(np.float(acc_val))

## 2. Datastore and Datasets

As an important part of experiment reproducibility, you'd like to separate your dataset from the training code and the development environment. There are two major ways that you can achieve this. First you can use datastores to define the connection to an Azure data store such as Blob, SQL or Databricks table, then you leverage Datasets to access the actual files and version the reference to those file assets.

### 2.1 Generate datastore references for the ML job

Once an ML Workspace is created, a storage account is created with a default container (a logical container that works as a folder - this is refered to as bucket in AWS S3). The container is attached to the Workspace automatically as the default storage account. You can find it as **workspaceblobstore** under Datastores (https://ml.azure.com). This storage account can be used for test and development but should not be used in production scenarios. Because if you decide to delete the Workspace, the default storage account is also deleted which results in losing your data. So it's wiser to create a separate storage account and attach it to the ML Workspace.

Here is how to access the default storage account:

In [ ]:
# get_default_datastore returns the default datastore attached to the Workspace

ds = ws.get_default_datastore()

# Retrieve other registered datastores
# ds = Datastore(ws, "generalpurposeaccount")

ds.name

In [ ]:
from azureml.core import Datastore

ds = Datastore(ws, "main_datastore")

As the name of the default datastore suggestion, it's a reference to a Blob storage. Blob storage is a general purpose data lake that can be used to store any type of binary, from image, to csv file. Here are other types of Azure Storage types that you can attach to the WorkSpace: (read [this](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#supported-data-storage-service-types) more info)

- Azure File Share
- Azure Data Lake
- Azure Data Lake Gen2
- Azure SQL Database
- Azure PostgreSQL
- Databricks File System

The following is an example of registering (attaching) a new Blob storage account:

In [ ]:
# from azureml.core.datastore import Datastore
# blob_datastore = Datastore.register_azure_blob_container(
#            workspace=ws,
#            datastore_name='<datastore_name>',
#            account_name='<account_name>', # Storage account name
#            container_name='<container_name>', # Name of Azure blob container
#            account_key='<account_key>')

Let's upload the files that I downloaded to this storage for longer retention. This datastore can be referenced later in my training job.

In [ ]:
ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

### 2.2 Datasets

There are two different types of Datasets, TabularDataset and FileDataset. The Tabular can be used to access tabular like datasources, such as csv, SQL, Databricks, etc which FileDatasets can be used for binary datasets such as image, audio, etc. The following is a way to define a tabular dataset from an online source.

Datasets are references to a datasource (registered under datasources or available over internet). In other words, they don't keep the data, they are the reference definitiosn. You can register the defined Datasets under the Datasets section (accessible through ml.azure.com). Each time you register the dataset under the same name, you'll get a new version generated. Later you can access a particular version for the registered Dataset. As the dataset doesn't store your data, if you remove or change the data, the Dataset object doesn't help you roll back the change. Therefore, it's recommended to keep the data untouched. 

In order to keep the actual dataset, you can copy the data (using Azure Data Factory SDK or AzCopy CLI tool or [Azure SDK](https://github.com/Azure/azure-sdk-for-python/tree/master/sdk/storage/azure-storage-blob)) to clone the data to a new blob location and keep the under a new Dataset object.

In the following cells, I've generated two versions of the Dataset. Every version of the titanic_ds can have different reference structure.

Now going back to our MNIST problem, let's create a Dataset object from the file uploaded to the default Datastore and register the Dataset:

In [ ]:
from azureml.core.dataset import Dataset

datastore_paths = [(ds, 'mnist/test-images.gz'),
 (ds, 'mnist/test-labels.gz'),
 (ds, 'mnist/train-images.gz'),
 (ds, 'mnist/train-labels.gz')]

mnist_dataset = Dataset.File.from_files(datastore_paths)
mnist_dataset

In [ ]:
mnist_dataset.register(workspace = ws,
                                 name = 'mnist_dataset',
                                 description = 'MNIST input dataset',
                                 create_new_version = True)

### Work with Tabular Files

Tabular files can be csv and parquet:

In [ ]:
! wget -P ./data https://dprepdata.blob.core.windows.net/demo/Titanic.csv

In [ ]:
import pandas as pd
pdf_titanic_csv = pd.read_csv('./data/Titanic.csv')

In [ ]:
ds.upload_files(['./data/Titanic.csv'], target_path='titanic', overwrite=True, show_progress=True)

In [ ]:
ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

In [ ]:
titanic_dataset = Dataset.Tabular.from_delimited_files(path = [(ds, 'titanic/Titanic.csv')])

In [ ]:
pdf_titanic = titanic_dataset.to_pandas_dataframe()
pdf_titanic.head()

In [ ]:
titanic_dataset = titanic_dataset.register(ws, name='Titanic Dataset')

At any time you can retrive a registered Dataset:

In [ ]:
titanic_ds_v1 = Dataset.get_by_name(workspace = ws,
                                 name = 'Titanic Dataset')

titanic_ds_v1.take(3).to_pandas_dataframe()

Now let's register it within the Datasets:

You define a new Dataset under **mnist_dataset** by accessing the actual file over the internet:

In [ ]:
web_paths = [
            'https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
            'https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
            'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
            'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz'
            ]

mnist_dataset_web = Dataset.File.from_files(path = web_paths)

mnist_dataset_web.register(workspace = ws,
                           name = 'mnist_dataset_web',
                           description = 'MNIST input dataset from web')

In [ ]:
mnist_dataset_v1 = Dataset.get_by_name(workspace = ws,
                                 name = 'mnist_dataset_web')

In [ ]:
# get the list of registered files back
mnist_dataset_v1.to_path()

# 3.1 Training on Remote Compute

In this section, we'd like to improve the previous local training experience by packaging the run in a more managed way so that the environment is also tracked and versioned.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "k80-2-g-2-n" # "v100-1-g-1-n"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Compute target not found...')

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)


In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())


In [ ]:
from azureml.core import Environment

tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.0-GPU')
tf_env 

In [ ]:
script_folder = './project-folder'
exp = Experiment(workspace=ws, name='tf-mnist-2')

#### Providing the dataset as an input parameter

One of the ways to make the training job flexible is to provide parameters instead of counting on hardcoded values. One of the important values is the data folder. As we've discussed the two ways of providing data to the ML Workspace (Datastores and Datasets) you can similarly provide both to your training job.

The as_mount function connects the dataset or datastore to the target compute at the run time, makes it look like accessing the files as if an external storage is mounted to the remote compute. This is very efficient, as it doesn't bring un-accessed data into the remote compute.

The other options is to call as_download function, but it requires the remote compute to fiest download the entire data (or the corresponding batch of data) to the local compute before running the script.

In [ ]:
mnist_dataset = Dataset.get_by_name(workspace = ws, name = 'mnist_dataset_web')

In [ ]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', mnist_dataset.as_mount(),
        '--batch-size', 64,
        '--first-layer-neurons', 256,
        '--second-layer-neurons', 128,
        '--learning-rate', 0.01]

src = ScriptRunConfig(source_directory=script_folder,
                      script='tf_mnist.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=tf_env)


In [ ]:
run = exp.submit(src)

In [ ]:
# 
run

In [ ]:
# # Returning a run from history
# from azureml.core import ScriptRun
# run = ScriptRun(exp, 'tf-mnist-2_1621465912_275dda41')

In [ ]:
RunDetails(run).show()


### AML Environments

Software dependency management is a common task for developers. You want to ensure that builds are reproducible without extensive manual software configuration. The Azure Machine Learning `Environment` class accounts for local development solutions such as pip and Conda and distributed cloud development through Docker capabilities.

To learn more, check out: [Create & use software environments in Azure Machine Learning
](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)

The Environment class is used to provide system configurations such as pip, and conda dependancies, base docker file or images and anything in between. You could use curated (already defined by Microsoft) environment or create a custom environment.

AML has several curated environment that could be used as is. Most of relevant depandancies are already installed so minimal changes is needed. To find the predefined environment you can run the following code:

In [ ]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print(f"Name: [{env}]")
        # print("packages", envs[env].python.conda_dependencies.serialize_to_string())


To create your custom environment, you can start with:

In [ ]:
from azureml.core.environment import Environment
my_custom_env = Environment(name="my_custom_environment")

# Or clone from a curated environment

tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.0-GPU')
tf_env_custom = tf_env.clone('Custom_AzureML-TensorFlow-2.0-GPU')

In [ ]:
tf_env_custom.python.conda_dependencies.add_pip_package("seaborn")

In [ ]:
tf_env_custom.python.conda_dependencies.serialize_to_string()

Finally you can register and retrieve it later for other workloads

In [ ]:
tf_env_custom = tf_env_custom.register(workspace=ws)

In [ ]:
envs = Environment.list(workspace=ws)
envs['Custom_AzureML-TensorFlow-2.0-GPU']

### Remote Job Flow

![Remote Job Flow](assets/RemoteJobOrchestration.png)

In [ ]:
run.wait_for_completion(show_output=True)

### Register the model into the Model Registry

The Run object can access to the output folder that is saved under the current Run within the experiment. By using the register_model function, you can register the model under the Model tab of the Workspace.

In [ ]:
run.register_model(model_name='tf-dnn-mnist-single-run', model_path='outputs/model')

### Environment Logging and Reproducibility

When the script is submitted using an Estimator object, Azure ML deploys a GPU base Linux docker image on the remote compute and based on the pip_packages and conda_packages parameters you can reproduce the environment as you wish.

Once the Run is submitted, the entire environment dependencies are logged and can be reproduced. Using the Run Details, you can explore the docker, conda and other dependencies.

In [ ]:
run.get_details()

### Training V2 - Hyperparameter tuning

In most of our DS experiments, in order to get to the optimal model, you have to try several hyper parameters on your algorithm. This can become super slow and time consuming. Using HyperDrive namespace, you can provide several choices to be passed to your training script through the parameters. In the example below, I provided three choices for the batch-size, 5 choices for the first layer neurons, 4 choices for the second-layer-neurons and a random value from the continuous space of log uniform distribution for the learning-rate.

Later by setting the max_total_runs as a parameter to the HyperDriveConfig, you can set the total number of tries. In the example below, it uses Random Sampling technique to find the next combination of hyperparameters to try. It will stop based on the BanditPolicy or if number of iterations reaches the max_total_runs.

Here are the types of sampling the hyperparamer space:
* Random sampling
* Grid sampling
* Bayesian sampling

Link: https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters

In case of having several nodes (in our case we only have 1 node), you can submit several parallel execution on multiple nodes by setting the max_concurrent_runs. For example, if you have set max_total_runs = 100 and set max_concurrent_runs to 20. Assuming that you have 20 nodes of GPU cluster, then there will be 20 concurrent runs on all of the 20 nodes. Therefore, in theory you require 5 cycles to compelete the entire 100 experiments.


![Hyper parameter tuning](assets/hyperparameter_tuning.jpg)

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, choice, loguniform

ps = RandomParameterSampling(
    {
        '--batch-size': choice(32, 64, 128),
        '--first-layer-neurons': choice(16, 64, 128, 256, 512),
        '--second-layer-neurons': choice(16, 64, 256, 512),
        '--learning-rate': loguniform(-6, -1)
    }
)


In [ ]:
args = ['--data-folder', mnist_dataset.as_mount()]

src = ScriptRunConfig(source_directory=script_folder,
                      script='tf_mnist.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=tf_env)

Next we will define an early `termnination policy`. This will terminate poorly performing runs automatically, reducing wastage of resources and instead efficiently using these resources for exploring other parameter configurations. In this example, we will use the TruncationSelectionPolicy, truncating the bottom performing 25% runs. It states to check the job every 2 iterations. If the primary metric (defined later) falls in the bottom 25% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.

In [ ]:
from azureml.train.hyperdrive import TruncationSelectionPolicy
policy = TruncationSelectionPolicy(evaluation_interval=2, truncation_percentage=25)


In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
htc = HyperDriveConfig(run_config=src, 
                       hyperparameter_sampling=ps, 
                       policy=policy, 
                       primary_metric_name='validation_acc', 
                       primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                       max_total_runs=4,
                       max_concurrent_runs=4)


In [ ]:
htr = exp.submit(config=htc)


In [ ]:
# # Returning a Hyper Drive run from history
# from azureml.train.hyperdrive import HyperDriveRun
# htr = HyperDriveRun(exp, 'HD_2bb4dc41-d40d-4f40-adba-b599202756d1')

In [ ]:
RunDetails(htr).show()


In [ ]:
htr.wait_for_completion(show_output=False)

In this run as you'll be having several models to collect. You can always get the **best** or collect the entire generated models. In the example below, the best model is retrieved and registered.

In [ ]:
best_run = htr.get_best_run_by_primary_metric()
print(best_run.get_file_names())

In [ ]:
model = best_run.register_model(model_name='tf-dnn-mnist-hyperp-tunning', model_path='outputs/model')

In [ ]:
model

### Training V3 - AutoML

Automated machine learning, also referred to as automated ML or AutoML, is the process of automating the time consuming, iterative tasks of machine learning model development. It allows data scientists, analysts, and developers to build ML models with high scale, efficiency, and productivity all while sustaining model quality. Automated ML in Azure Machine Learning is based on a breakthrough from our Microsoft Research division.

To learn more, please visit: [What is automated machine learning (AutoML)?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml)

Now, let's try to train model using Azure ML AutoML:

In [ ]:
import pandas as pd
df_train = pd.DataFrame(data=X_train)
df_train['target'] = y_train

df_train.to_csv('./data/mnist_train.csv')

ds.upload_files(['./data/mnist_train.csv'], target_path='mnist_numpy', overwrite=True, show_progress=True)

In [ ]:
df_train.shape

In [ ]:
from azureml.core.dataset import Dataset

mnist_dataset = Dataset.Tabular.from_delimited_files(path = [(ds, 'mnist_numpy/mnist_train.csv')])

In [ ]:
# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
    print(compute_target.get_status().serialize())
except ComputeTargetException:
    print('Compute target not found...')

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient


automl_settings = {
    "experiment_timeout_hours" : 10,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    #"n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             experiment_exit_score = 0.9984,
                             blocked_models = ['KNN','LinearSVM'],
                             enable_onnx_compatible_models=True,
                             training_data = mnist_dataset,
                             label_column_name = 'target',
                             **automl_settings
                            )


In [ ]:
experiment=Experiment(ws, 'automl_mnist')

In [ ]:
remote_run = experiment.submit(automl_config, show_output = False)


In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()


In [ ]:
remote_run.wait_for_completion(show_output=False)

### Register the best model based on the primary metric

In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run.register_model(model_name = 'automl_mnist', model_path='outputs')

In [ ]:
model = Model(ws, 'automl_mnist')

## MLFlow

In [ ]:
import sys, os
import mlflow
import mlflow.azureml

In [ ]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [ ]:
experiment_name = "at-rogers"
mlflow.set_experiment(experiment_name)

In [ ]:
experiment_name = "pytorch-with-mlflow"
mlflow.set_experiment(experiment_name)

In [ ]:
with mlflow.start_run():
    mlflow.log_metric("metric_mlflow", 1.1)
    mlflow.log_metric("metric_mlflow", 3)
    mlflow.log_metric("metric_mlflow", 3.2)
    mlflow.log_metric("metric_mlflow", 2.2)
    mlflow.log_metric("metric_mlflow", 8.1)


In [ ]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="AzureML-PyTorch-1.4-GPU").clone("mlflow-env")

env.python.conda_dependencies.add_pip_package("azureml-mlflow")
env.python.conda_dependencies.add_pip_package("Pillow==6.0.0")

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="./mlflow", script="train.py")
src.run_config.environment = env
src.run_config.target = "gpu-cluster"

In [ ]:
from azureml.core import Experiment

exp = Experiment(ws, experiment_name)
run = exp.submit(src)

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.register_model('MLFlow', model_path='model/')

In [ ]:
experiment_name = "keras-with-mlflow"
mlflow.set_experiment(experiment_name)

In [ ]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="AzureML-TensorFlow-2.1-GPU").clone("mlflow-env")

env.python.conda_dependencies.add_pip_package("azureml-mlflow")
env.python.conda_dependencies.add_pip_package("keras==2.3.1")
env.python.conda_dependencies.add_pip_package("numpy")

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="./mlflow", script="train_autolog.py")
src.run_config.environment = env
src.run_config.target = "gpu-clu-custom"

In [ ]:
from azureml.core import Experiment

exp = Experiment(ws, experiment_name)
run = exp.submit(src)